In [7]:
import numpy as np
import boto3
import requests

In [16]:
# Initiate AWS SSM integration for secrets storage
ssm = boto3.client('ssm')

# Grab api key from AWS and authenticate with google
google_api_key = ssm.get_parameter(Name="GOOGLE_CLOUD_API_KEY", WithDecryption=True)["Parameter"]["Value"]

# Prompt user for desired mode of transit
mode = "driving"

# Define constraints for creating distance matrix - maximum 100 elements per query based on num origins and dests
max_elements = 100
num_addresses = len(place_id_list)
rows_per_send = max_elements // num_addresses

# Evaluate number of queries required
num_queries, remaining_rows = divmod(num_addresses, max_elements)

# Create distance matrix with Distance Matrix API - https://developers.google.com/maps/documentation/distance-matrix
distance_matrix = np.empty((num_addresses, num_addresses))

In [17]:
def query_dist_matrix(payload):
    """
    Queries google's distance matrix API
    :param payload: Contains parameters for query
    :return: dist matrix values as list if successful
    """
    url = "https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial"

    try:
        response = requests.request("GET", url, data=payload)

    except requests.exceptions.RequestException:
        print("Unable to validate address. Please try again.")
        return 0

    return response

In [18]:
def build_dist_matrix(response, num_addresses, rows):
    """
    Takes response from distance matrix API to add rows to the distance matrix. We will use travel time rather than
    distance.
    :param response: response from Google's distance matrix API
    :param num_addresses: Number of addresses to set array size for columns
    :param rows: Number of rows being included in response
    :return: row for distance matrix
    """
    distance_matrix = np.empty((num_addresses, rows))

    for row in response['rows']:
        # Loop through each element in the rows of the response and populate a list of each value
        row_list = [row['elements'][i]['duration']['value'] for i in range(len(row["elements"]))]

        # Add row list to the array
        distance_matrix = distance_matrix + np.array(row_list)

    return distance_matrix

In [21]:
place_id_list = ['3610+Hacks+Cross+Rd+Memphis+TN', # depot
                       '1921+Elvis+Presley+Blvd+Memphis+TN',
                       '149+Union+Avenue+Memphis+TN',
                       '1034+Audubon+Drive+Memphis+TN',
                       '1532+Madison+Ave+Memphis+TN',
                       '706+Union+Ave+Memphis+TN',
                       '3641+Central+Ave+Memphis+TN',
                       '926+E+McLemore+Ave+Memphis+TN',
                       '4339+Park+Ave+Memphis+TN',
                       '600+Goodwyn+St+Memphis+TN',
                       '2000+North+Pkwy+Memphis+TN',
                       '262+Danny+Thomas+Pl+Memphis+TN',
                       '125+N+Front+St+Memphis+TN',
                       '5959+Park+Ave+Memphis+TN',
                       '814+Scott+St+Memphis+TN',
                       '1005+Tillman+St+Memphis+TN'
                      ]

# Send sets of addresses in chunks equal to num_queries
for i in range(num_queries):
    origin_addresses = place_id_list[i * num_queries: (i+1) * num_queries]

    # Turn lists into strings separated by pipe to be recognized by API
    payload = {
        "origins": " ".join(origin_addresses).replace(" ", "|"),
        "destinations": " ".join(place_id_list).replace(" ", "|"),
        "mode": mode,
        "key": google_api_key
    }

    response = query_dist_matrix(payload)
    distance_matrix += build_dist_matrix(response, num_addresses, rows_per_send)

    distance_matrix

    # Send remaining rows
    for i in range(remaining_rows):
        origin_addresses = place_id_list[rows_per_send * num_queries: rows_per_send * num_queries + remaining_rows]
        payload = {
            "origins": " ".join(origin_addresses).replace(" ", "|").replace("+", "%2B"),
            "destinations": " ".join(place_id_list).replace(" ", "|").replace("+", "%2B"),
            "mode": mode,
            "key": google_api_key
        }

        response = query_dist_matrix(payload)

        distance_matrix += build_dist_matrix(response, num_addresses, remaining_rows)

distance_matrix


array([[ 1.00132568e-307,  1.33508676e-307,  1.11258022e-307,
         5.78535840e-307,  6.87198057e+010,  8.62437526e-288,
         4.42871215e+088,  2.40900058e-240,  3.33583159e+077,
         3.96731406e-245,  1.13555857e-250,  2.93551169e-077,
         1.13625240e-298,  9.65059984e-072,  8.62006703e-043,
         9.16854595e-072],
       [ 5.63962981e-062,  1.03516115e-230,  2.05184377e+005,
         5.95097414e-071,  8.62437521e-288,  9.73132484e+015,
         4.31438379e-294,  1.08390204e+016,  3.32874673e+101,
        -3.90036023e-298, -5.73714408e-279,  1.42936525e-284,
        -3.82270250e-298, -1.23418316e-178,  5.32861362e-079,
        -1.34895495e-198],
       [ 9.08597309e-159, -5.36912481e+147,  2.12858218e-051,
         4.65625919e-191,  1.42363284e+039,  7.72345727e+136,
         3.28082517e+236, -5.26843104e+132, -6.88032632e+273,
        -1.72764753e-113,  3.52751762e+213,  2.14146416e-007,
        -3.14619583e+172,  3.86876563e+058,  1.05401808e+063,
         2.74265